In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# membaca file Excel
df = pd.read_excel('/content/drive/MyDrive/prosaindata/tugas/PTA.xlsx')

# melakukan pembersihan data pada kolom Abstrak
df['Abstrak'] = df['Abstrak'].str.replace('[^\w\s]','', regex=True)

# menyimpan hasil ke file Excel baru
df.to_excel('Pta_cleaned.xlsx', index=False)


In [26]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# download stop words dan stemmer
nltk.download('stopwords')
nltk.download('punkt')

# membaca file Excel
df = pd.read_excel('/content/Pta_cleaned.xlsx')

# menghapus stop words
stop_words = set(stopwords.words('indonesian'))
df['Abstrak'] = df['Abstrak'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# stemming
stemmer = PorterStemmer()
df['Abstrak'] = df['Abstrak'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

# tokenization
df['Abstrak'] = df['Abstrak'].apply(lambda x: word_tokenize(x))

# menyimpan hasil ke file Excel baru
df.to_excel('pta_preprocessed.xlsx', index=False)
data = pd.read_excel('pta_preprocessed.xlsx')
print(data.columns)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Index(['NPM', 'Judul', 'Abstrak', 'Prodi', 'Label'], dtype='object')


In [47]:
import pandas as pd
import numpy as np
import nltk
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# download punkt tokenizer dari nltk
nltk.download('punkt')

# membaca file csv
df = pd.read_excel('/content/pta_preprocessed.xlsx')

# tokenisasi setiap kalimat dalam abstrak
sentences = [nltk.word_tokenize(abstrak) for abstrak in df['Abstrak']]

# train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Transformasi corpus menjadi vektor
vectors = model.wv

# Lakukan reduksi dimensi pada vektor menggunakan PCA
pca = PCA(n_components=50)
vectors_pca = pca.fit_transform(vectors.vectors)

# Sesuaikan ukuran df['Label'] dengan vectors_pca
df = df.iloc[:vectors_pca.shape[0], :]

# Pisahkan data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(vectors_pca, df['Label'], test_size=0.2)

# Klasifikasi dengan naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print("Akurasi klasifikasi: {:.2f}%".format(score * 100))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: ignored

In [18]:
import pandas as pd
from sklearn.decomposition import PCA

# membaca file csv hasil Word2Vec
vectors = pd.read_csv('/content/abstract_vectors.csv', index_col=0)

# membuat objek PCA dengan 100 komponen
pca = PCA(n_components=100)

# melakukan fit transform pada vektor
vectors_pca = pca.fit_transform(vectors)

# menyimpan hasil reduksi dimensi ke dalam file csv
vectors_pca = pd.DataFrame(vectors_pca, index=vectors.index)
vectors_pca.to_csv('pta_vectors_pca.csv')
# load dataset
data = pd.read_csv('pta_vectors_pca.csv')

# cek kolom pada dataset
print(data.columns)



Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'],
      dtype='object', length=101)


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# membaca file csv
df = pd.read_csv('/content/pta_vectors_pca.csv', index_col=0)

# memisahkan kolom label sebagai target
target = df.index.values
data = df.values

# membagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# inisialisasi model Naive Bayes
nb = GaussianNB()

# training model dengan data train
nb.fit(X_train, y_train)

# membuat prediksi dengan data test
y_pred = nb.predict(X_test)

# menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)


Akurasi: 0.0
